In [1]:
import gensim
import re
import logging
import numpy as np
import pandas as pd
import multiprocessing

from re import sub
from time import time 
from unidecode import unidecode
from gensim.models import Word2Vec
from collections import defaultdict
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.models.phrases import Phrases, Phraser

logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [10]:
file_model = pd.read_csv('musk_clean.csv')


AttributeError: 'Series' object has no attribute 'join'

In [3]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

INFO - 15:35:28: Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2023-05-20T15:35:28.480117', 'gensim': '4.3.1', 'python': '3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) \n[Clang 13.0.0 (clang-1300.0.29.3)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'created'}
INFO - 15:35:28: collecting all words and their counts
WARNING - 15:35:28: Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
INFO - 15:35:28: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 15:35:29: PROGRESS: at sentence #50000, processed 18984775 words, keeping 76 word types
INFO - 15:35:29: collected 76 word types from a corpus of 23355499 raw words and 61049 sentences
INFO - 15:35:29: Creating a fresh vocabulary
INFO - 15:35:29: Word2Vec lifecycle event {'msg': 'effective_min_count=3 retains 73 unique words (96.05% of original 76, drops 3)', 'datetime': '2023-05-20T1

Time to build vocab: 0.02 mins


In [4]:
start = time()

w2v_model.train(sentences,
                total_examples=w2v_model.corpus_count,
                epochs=30,
                report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

INFO - 15:35:29: Word2Vec lifecycle event {'msg': 'training model with 7 workers on 73 vocabulary and 300 features, using sg=0 hs=0 sample=1e-05 negative=20 window=4 shrink_windows=True', 'datetime': '2023-05-20T15:35:29.627826', 'gensim': '4.3.1', 'python': '3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) \n[Clang 13.0.0 (clang-1300.0.29.3)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'train'}
INFO - 15:35:30: EPOCH 0 - PROGRESS: at 57.72% examples, 230529 words/s, in_qsize 8, out_qsize 9
INFO - 15:35:31: EPOCH 0: training on 23355499 raw words (399069 effective words) took 1.7s, 233040 effective words/s
INFO - 15:35:32: EPOCH 1 - PROGRESS: at 58.89% examples, 235552 words/s, in_qsize 14, out_qsize 11
INFO - 15:35:33: EPOCH 1: training on 23355499 raw words (399062 effective words) took 1.7s, 237997 effective words/s
INFO - 15:35:34: EPOCH 2 - PROGRESS: at 58.23% examples, 232714 words/s, in_qsize 8, out_qsize 5
INFO - 15:35:34: EPOCH 2: training on 23355499 raw words (39919

Time to train the model: 0.85 mins


In [5]:
w2v_model.save("word2vec.model")

INFO - 15:36:21: Word2Vec lifecycle event {'fname_or_handle': 'word2vec.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-05-20T15:36:21.436715', 'gensim': '4.3.1', 'python': '3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) \n[Clang 13.0.0 (clang-1300.0.29.3)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'saving'}
INFO - 15:36:21: not storing attribute cum_table
INFO - 15:36:21: saved word2vec.model


In [6]:
from sklearn.cluster import KMeans
word_vectors = Word2Vec.load("word2vec.model").wv

model = KMeans(n_clusters=2,
               max_iter=1000,
               random_state=True,
               n_init=50)
model.fit(X=word_vectors.vectors.astype('double'))

INFO - 15:36:24: loading Word2Vec object from word2vec.model
INFO - 15:36:24: loading wv recursively from word2vec.model.wv.* with mmap=None
INFO - 15:36:24: setting ignored attribute cum_table to None
INFO - 15:36:24: Word2Vec lifecycle event {'fname': 'word2vec.model', 'datetime': '2023-05-20T15:36:24.619527', 'gensim': '4.3.1', 'python': '3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) \n[Clang 13.0.0 (clang-1300.0.29.3)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'loaded'}


KMeans(max_iter=1000, n_clusters=2, n_init=50, random_state=True)

In [7]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)

[('F', 0.9917163252830505),
 ('N', 0.9905520081520081),
 ('M', 0.9858402609825134),
 ('G', 0.9855461120605469),
 ('X', 0.9844103455543518),
 ('Z', 0.9827008843421936),
 ('W', 0.9820296764373779),
 ('J', 0.9799992442131042),
 ('H', 0.9799615740776062),
 ('Q', 0.9798189997673035)]